# Text2Img

Contains modules for doing basic text2img and img2img inference with stable diffusion models.

In [ ]:
#| default_exp gen

In [ ]:
#| export
import matplotlib.pyplot as plt
import torch
import gc

from pathlib import Path
from PIL import Image
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers import LMSDiscreteScheduler
from torch import tensor
from torchvision import transforms
from tqdm import tqdm
from IPython import display
import fastcore.all as fc

In [ ]:
#| export
def get_device():
    if torch.cuda.is_available(): device = 'cuda'
    else: device = 'cpu'
    return device

In [ ]:
#| export
def encode_text(prompt, device, tok=None, te=None):
    if tok is not None: tokenizer = tok
    if te is not None: text_encoder = te
    tokens = tokenizer(
        prompt, 
        padding="max_length", 
        max_length=tokenizer.model_max_length, 
        truncation=True, 
        return_tensors="pt"
    )
    
    embedding = text_encoder(
        tokens.input_ids.to(device)
    )[0].half()
    
    return embedding

In [ ]:
#| export
def encode_img(file_path, height, width, device, autoencoder=None):
    if autoencoder is not None: vae = autoencoder
    im = Image.open(file_path).resize((height, width))
    im_tensor = transforms.ToTensor()(im).unsqueeze(0).to(device).half()
    init_latents = 0.18215 * vae.encode(im_tensor * 2 - 1).latent_dist.sample()
    return init_latents

In [ ]:
#| export
def decode_img(latent, autoencoder=None):
    if autoencoder is not None: vae = autoencoder
    with torch.no_grad():
        img = vae.decode(1 / 0.18215 * latent).sample

    img = (img / 2 + 0.5).clamp(0, 1)
    img = img.detach().cpu().permute(0, 2, 3, 1).float().numpy()
    img = (img * 255).round().astype("uint8")
    return Image.fromarray(img[0])

In [ ]:
#| export
def get_diffusion_pipe(hub_path):
    device = get_device()
    unet = UNet2DConditionModel.from_pretrained(hub_path, subfolder="unet", torch_dtype=torch.float16).to(device);
    vae = AutoencoderKL.from_pretrained(hub_path, subfolder='vae', torch_dtype=torch.float16).to(device);
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to(device);
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16);
    return unet, vae, text_encoder, tokenizer

In [ ]:
#| export
def get_scheduler(beta_start=0.00085, beta_end=0.012, **kwargs):
    return LMSDiscreteScheduler(
        beta_start=beta_start, 
        beta_end=beta_end, 
        beta_schedule="scaled_linear", 
        num_train_timesteps=1000,
        **kwargs
    )

In [ ]:
#| export
def generate(prompt, img_path, guidance_scale=7.5, seed=None, steps=70, height=512, width=512, start_step=10, return_latents=False, device='cuda', autoencoder=None, uunet=None, tok=None, sched=None, te=None):
    
    if uunet is not None: unet = uunet
    if sched is not None: scheduler = sched
    text = encode_text(prompt, device, tok=tok, te=te)
    uncond = encode_text("", device, tok=tok, te=te)
    text_embeddings = torch.cat([uncond, text])
    
    if seed: torch.manual_seed(seed)
    
    scheduler.set_timesteps(steps)
    
    if img_path is not None:
        init_latents = encode_img(img_path, height, width, device, autoencoder=autoencoder)
        noise = torch.randn([1, unet.in_channels, height//8, width//8]).to(device)
        latents = scheduler.add_noise(
            init_latents, 
            noise, 
            timesteps=torch.tensor([scheduler.timesteps[start_step]])
        ).to(device).half()
    else: 
        latents = torch.randn(1, unet.in_channels, height//8, width//8).to(device).half()
        latents = latents * scheduler.init_noise_sigma
    
    for i, ts in enumerate(tqdm(scheduler.timesteps)):
        
        if i > start_step:
            latent_model_input = torch.cat([latents] * 2)
            latent_model_input = scheduler.scale_model_input(latent_model_input, ts)
        
            with torch.no_grad(): 
                noise_pred = unet(latent_model_input, ts, encoder_hidden_states=text_embeddings).sample
                
            u, t = noise_pred.chunk(2)
            noise_pred = u + guidance_scale * (t - u)
            latents = scheduler.step(noise_pred, ts, latents).prev_sample
        
    image = decode_img(latents, autoencoder=autoencoder)
    
    if return_latents: return image, latents
    else: return image

In [ ]:
def embed_clip(embed_path, text_encoder, tokenizer, token=None):
    loaded_learned_embeds = torch.load(embed_path, map_location="cpu")

    # separate token and the embeds
    trained_token = list(loaded_learned_embeds.keys())[0]
    embeds = loaded_learned_embeds[trained_token]

    # cast to dtype of text_encoder
    dtype = text_encoder.get_input_embeddings().weight.dtype
    embeds.to(dtype)

    # add the token in tokenizer
    token = token if token is not None else trained_token
    num_added_tokens = tokenizer.add_tokens(token)
    if num_added_tokens == 0:
        raise ValueError(f"The tokenizer already contains the token {token}. Please pass a different `token` that is not already in the tokenizer.")

    # resize the token embeddings
    text_encoder.resize_token_embeddings(len(tokenizer))

    # get the id for the token and assign the embeds
    token_id = tokenizer.convert_tokens_to_ids(token)
    text_encoder.get_input_embeddings().weight.data[token_id] = embeds

In [ ]:
#| export
class Wizard:
    def __init__(self, model_path, embeddings:list=None):
        self.unet, self.vae, self.text_encoder, self.tokenizer = get_diffusion_pipe(model_path)
        self.scheduler = get_scheduler()
        self.device = get_device()
        if embeddings is not None:
            for e in embeddings: embed_clip(e, self.text_encoder, self.tokenizer)
    def gen(self, prompt, img_path=None, guidance_scale=7.5, seed=None, steps=70, height=512, width=512, start_step=0, return_latents=False):
        if img_path is not None and start_step == 0: start_step = 10
        gen = generate(
            prompt, 
            img_path=img_path, 
            guidance_scale=guidance_scale, 
            seed=seed, 
            steps=steps, 
            height=height, 
            width=width, 
            start_step=start_step, 
            return_latents=return_latents, 
            device=self.device,
            sched=self.scheduler,
            autoencoder=self.vae,
            tok=self.tokenizer,
            uunet=self.unet,
            te=self.text_encoder
        )
        if return_latents: 
            img, latents = gen
            return img, latents
        else: return gen

In [ ]:
#| export
import nbdev; nbdev.nbdev_export()